In [0]:
import h5py
import random
class Miir():
    def __init__(self, data_path='/home/arjun/PycharmProjects/untitled3/openmiir-rl-2016/EEG-Disetangled-Classification-master/OpenMIIR-Perception-512Hz.hdf5', train=True):
        h5 = h5py.File(data_path, 'r')
        self.train = train
        features = h5['features']
        targets = h5['targets']
        subjects = h5['subjects']
        train_indx=[]
        self.train_features=[]
        self.test_features=[]
        self.train_targets = []
        self.test_targets = []
        self.train_subjects = self.test_subjects=[]
        self.test_subject_id = random.randint(0,8)
        for i,e in enumerate(subjects):
            if e != self.test_subject_id:
                train_indx.append(i)
        for i, e in enumerate(features):
            if i in train_indx:
                self.train_features.append(e)
            if i not in train_indx:
                self.test_features.append(e)
        for i, e in enumerate(targets):
            if i in train_indx:
                self.train_targets.append(e)
            if i not in train_indx:
                self.test_targets.append(e)
        for i, e in enumerate(subjects):
            if i in train_indx:
                self.train_subjects.append(e)
            if i not in train_indx:
                self.test_subjects.append(e)
        self.train_size = len(self.train_features)
        self.test_size = len(self.test_features)
    
    def __len__(self):
        if self.train:
            return self.train_size
        else:
            return self.test_size

    def __getitem__(self, index):
        if self.train:
            return self.train_features[index], self.train_targets[index], self.train_subjects[index]
        else:
            return self.test_features[index], self.test_targets[index], self.test_subjects[index]

pp = Miir()
import numpy as np
work = np.squeeze(np.array(pp.train_features))
work_test = np.squeeze(np.array(pp.test_features))
y_train = np.array(pp.train_targets)
y_test = np.array(pp.test_targets)


In [1]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 5
num_classes = 12
epochs = 12


Using TensorFlow backend.


In [0]:
img_rows, img_cols = 512, 512


In [0]:
import numpy as np
preds_test = np.load('/content/drive/My Drive/preds_test.npy')
preds = np.load('/content/drive/My Drive/preds.npy')
work_test = np.load('/content/drive/My Drive/work_test.npy')
work = np.load('/content/drive/My Drive/work.npy')

### Define the model

The following example uses a standard conv-net that has 3 layers with drop-out and batch normalization between each layer.

In [4]:
x_train = work.reshape(work.shape[0], img_rows, img_cols, 1)
x_test = preds.reshape(preds.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
print(x_train.shape)

(480, 512, 512, 1)


In [0]:
y_train = np.load('train_target.npy')
y_test = np.load('test_target.npy')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [11]:
x_test= work_test.reshape(work_test.shape[0], img_rows, img_cols, 1)
model.fit(x_train, y_train,
          batch_size=32,
          epochs=25,
          verbose=1,callbacks=[TestCallback((x_test))])


Epoch 1/25
480/480 [==============================] - 92s 193ms/step - loss: 19.5151 - accuracy: 0.0854

Testing loss: 2.449952713648478, acc: 0.11666666716337204

Epoch 2/25
480/480 [==============================] - 89s 186ms/step - loss: 2.2835 - accuracy: 0.2167

Testing loss: 2.4375569343566896, acc: 0.10000000149011612

Epoch 3/25
480/480 [==============================] - 88s 184ms/step - loss: 2.0631 - accuracy: 0.3021

Testing loss: 2.418277374903361, acc: 0.0833333358168602

Epoch 4/25
480/480 [==============================] - 90s 187ms/step - loss: 1.9898 - accuracy: 0.3333

Testing loss: 2.412712812423706, acc: 0.1666666716337204

Epoch 5/25
480/480 [==============================] - 90s 188ms/step - loss: 1.7243 - accuracy: 0.3938

Testing loss: 2.368067423502604, acc: 0.11666666716337204

Epoch 6/25
480/480 [==============================] - 91s 190ms/step - loss: 1.5203 - accuracy: 0.4958

Testing loss: 2.369289493560791, acc: 0.11666666716337204

Epoch 7/25
480/480 [==